## <span style="font-family: Arial; font-weight:bold;font-size:1.9em;color:#0e92ea">Assignment 3: Bancassurance</span>

&nbsp;

<p align="center" style="font-family: Arial;color:#0e92ea;font-size:1em;">
Build a model to identify the positively responding customers who have a higher probability of purchasing the insurance
    
</p>

##  <span style="font-family: Arial; font-weight:bold;font-size:1.9em;color:#0e92ea"> Contents:</span>

<ol style="font-family: Arial;color:#0e92ea;font-size:1em;">
    <li>Data Overview and EDA</li>
    <li>Outlier Treatment and Feature Engineering</li>
    <li>Building Decision Tree Model and Analysis</li>
    <li>Building Logistic Regression Model and Analysis</li>
    <li>Conclusions</li>
</ol>

In [1]:
import os
import warnings
import pprint
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from datetime import datetime
from matplotlib import gridspec
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn import tree
from os import system
from IPython.display import Image


warnings.filterwarnings('ignore')

main_data = pd.read_csv('My Bank.csv')

default_color_palette  = ["#03A9F4","#7C4DFF", "#FF5252", "#D50000", "#FF6F00", "#0288D1", "#7C4DFF"] 
stats_colors           = {'Mean':'#D50000', 'Mode':'#FF3D00', 'Median':'#2962FF'} # Set standard colors for mean, mode and median to use accross entire notebook.
gender_color_palette   = {"Female":"#E91E63", "Male":"#42A5F5"} # Gender based colors.

####  <span style="font-family: Arial; font-weight:bold;font-size:1.9em;color:#0e92ea"> 1. Data Overview and EDA:
    
<span style="font-family: Arial; font-weight:bold;font-size:1em;color:#0e92ea"> General Observations:
<ul>
    <li>The data has no mising values.</li>
</ul>

In [2]:
main_data.head(10)

,CUST_ID,TARGET,AGE,GENDER,BALANCE,OCCUPATION,SCR,HOLDING_PERIOD,ACC_TYPE,ACC_OP_DATE,...,NO_OF_CHQ_DR_TXNS,FLG_HAS_CC,AMT_ATM_DR,AMT_BR_CSH_WDL_DR,AMT_CHQ_DR,AMT_NET_DR,AMT_MOB_DR,FLG_HAS_ANY_CHGS,FLG_HAS_NOMINEE,FLG_HAS_OLD_LOAN
0,C7927,0,27,M,3383.75,SELF-EMP,776,30,SA,3/23/2005,...,0,0,13100,0,0,973557.0,0,0,1,1
1,C6877,0,47,M,287489.04,SAL,324,28,SA,10-11-2008,...,0,0,6600,0,0,799813.0,0,1,1,0
2,C19922,0,40,M,18216.88,SELF-EMP,603,2,SA,4/26/2012,...,2,0,11200,561120,49320,997570.0,0,1,1,1
3,C8183,0,53,M,71720.48,SAL,196,13,CA,07-04-2008,...,4,0,26100,673590,60780,741506.0,71388,0,1,0
4,C12123,0,36,M,1671622.89,PROF,167,24,SA,12/29/2001,...,0,0,0,808480,0,0.0,0,0,1,0
5,C257,0,42,F,521685.69,PROF,493,26,SA,06-07-2001,...,0,1,18500,379310,0,0.0,0,0,1,1
6,C8300,0,30,M,204458.60,PROF,479,14,SA,8/25/2002,...,4,0,6200,0,10580,770065.0,0,1,0,1
7,C5999,0,53,F,58516.34,SAL,562,25,CA,3/15/2009,...,0,0,0,945160,0,0.0,0,0,1,1
8,C5692,0,42,F,13158.14,SELF-EMP,105,15,SA,02-08-2010,...,1,1,35400,198430,51490,326421.0,170332,0,1,1
9,C3241,0,30,M,831150.18,PROF,170,13,SA,03-02-2008,...,4,0,18000,869880,32610,0.0,0,0,0,0


In [3]:
# Drop The Id Column
main_data.drop("CUST_ID", axis=1, inplace=True)
main_data.head(10)

,TARGET,AGE,GENDER,BALANCE,OCCUPATION,SCR,HOLDING_PERIOD,ACC_TYPE,ACC_OP_DATE,LEN_OF_RLTN_IN_MNTH,...,NO_OF_CHQ_DR_TXNS,FLG_HAS_CC,AMT_ATM_DR,AMT_BR_CSH_WDL_DR,AMT_CHQ_DR,AMT_NET_DR,AMT_MOB_DR,FLG_HAS_ANY_CHGS,FLG_HAS_NOMINEE,FLG_HAS_OLD_LOAN
0,0,27,M,3383.75,SELF-EMP,776,30,SA,3/23/2005,146,...,0,0,13100,0,0,973557.0,0,0,1,1
1,0,47,M,287489.04,SAL,324,28,SA,10-11-2008,104,...,0,0,6600,0,0,799813.0,0,1,1,0
2,0,40,M,18216.88,SELF-EMP,603,2,SA,4/26/2012,61,...,2,0,11200,561120,49320,997570.0,0,1,1,1
3,0,53,M,71720.48,SAL,196,13,CA,07-04-2008,107,...,4,0,26100,673590,60780,741506.0,71388,0,1,0
4,0,36,M,1671622.89,PROF,167,24,SA,12/29/2001,185,...,0,0,0,808480,0,0.0,0,0,1,0
5,0,42,F,521685.69,PROF,493,26,SA,06-07-2001,192,...,0,1,18500,379310,0,0.0,0,0,1,1
6,0,30,M,204458.60,PROF,479,14,SA,8/25/2002,177,...,4,0,6200,0,10580,770065.0,0,1,0,1
7,0,53,F,58516.34,SAL,562,25,CA,3/15/2009,99,...,0,0,0,945160,0,0.0,0,0,1,1
8,0,42,F,13158.14,SELF-EMP,105,15,SA,02-08-2010,88,...,1,1,35400,198430,51490,326421.0,170332,0,1,1
9,0,30,M,831150.18,PROF,170,13,SA,03-02-2008,111,...,4,0,18000,869880,32610,0.0,0,0,0,0


In [4]:
print(f"Rows : {main_data.shape[0]} x Columns : {main_data.shape[1]}")

Rows : 20000 x Columns : 25


In [6]:
'''
Input:
Pandas DataFrame

Output:
Displays DataFrame structure 
(columns, nulls and non nulls counts and percentage highlighing columns with most number of nulls)

Retunrs:
N/A
'''
def info(dataFrame):
    nulls_series             = dataFrame.isna().sum()         # Get a series counting number of empty values for each column
    nonnulls_series          = dataFrame.notnull().sum()      # Get a series counting number of non empty valuesfor each column 
    nulls_percentage         = ((nulls_series * 100)/(nulls_series + nonnulls_series)).astype(float)
    column_datatypes         = dataFrame.dtypes               # Get a series containing data types for each column 

    nulls_count_df = pd.concat(
        [nulls_series, nonnulls_series, nulls_percentage, column_datatypes],
        axis=1,
        keys=["Nulls", "Non-Nulls","Nulls %", "Type"], 
        sort=True)
    cm = sns.light_palette("red", as_cmap=True)
    display(nulls_count_df.style.background_gradient(cmap=cm, subset=pd.IndexSlice[:, ['Nulls %']]).format(formatter={('Nulls %'): "{:.2f}%"}))    

In [7]:
info(main_data)

,Nulls,Non-Nulls,Nulls %,Type
ACC_OP_DATE,0,20000,0.00%,object
ACC_TYPE,0,20000,0.00%,object
AGE,0,20000,0.00%,int64
AMT_ATM_DR,0,20000,0.00%,int64
AMT_BR_CSH_WDL_DR,0,20000,0.00%,int64
AMT_CHQ_DR,0,20000,0.00%,int64
AMT_MOB_DR,0,20000,0.00%,int64
AMT_NET_DR,0,20000,0.00%,float64
BALANCE,0,20000,0.00%,float64
FLG_HAS_ANY_CHGS,0,20000,0.00%,int64


In [9]:
main_data.nunique()

TARGET                         2
AGE                           35
GENDER                         3
BALANCE                     9924
OCCUPATION                     4
SCR                          896
HOLDING_PERIOD                31
ACC_TYPE                       2
ACC_OP_DATE                 4869
LEN_OF_RLTN_IN_MNTH          193
NO_OF_L_CR_TXNS               76
NO_OF_BR_CSH_WDL_DR_TXNS      16
NO_OF_ATM_DR_TXNS             26
NO_OF_NET_DR_TXNS             17
NO_OF_MOB_DR_TXNS              6
NO_OF_CHQ_DR_TXNS             16
FLG_HAS_CC                     2
AMT_ATM_DR                   646
AMT_BR_CSH_WDL_DR           7369
AMT_CHQ_DR                  4891
AMT_NET_DR                  4847
AMT_MOB_DR                  2312
FLG_HAS_ANY_CHGS               2
FLG_HAS_NOMINEE                2
FLG_HAS_OLD_LOAN               2
dtype: int64

In [10]:
main_data[main_data.select_dtypes(include=['object']).columns.tolist()].head(10)

,GENDER,OCCUPATION,ACC_TYPE,ACC_OP_DATE
0,M,SELF-EMP,SA,3/23/2005
1,M,SAL,SA,10-11-2008
2,M,SELF-EMP,SA,4/26/2012
3,M,SAL,CA,07-04-2008
4,M,PROF,SA,12/29/2001
5,F,PROF,SA,06-07-2001
6,M,PROF,SA,8/25/2002
7,F,SAL,CA,3/15/2009
8,F,SELF-EMP,SA,02-08-2010
9,M,PROF,SA,03-02-2008


In [12]:
main_data.describe().T

,count,mean,std,min,25%,50%,75%,max
TARGET,20000.0,0.125600,0.331406,0.0,0.000,0.000,0.00,1.00
AGE,20000.0,38.418150,9.577985,21.0,30.000,38.000,46.00,55.00
BALANCE,20000.0,511362.192118,723592.961792,0.0,64754.025,231675.845,653876.85,8360430.57
SCR,20000.0,440.150300,249.887582,100.0,227.000,364.000,644.00,999.00
HOLDING_PERIOD,20000.0,14.955650,8.642634,1.0,7.000,15.000,22.00,31.00
LEN_OF_RLTN_IN_MNTH,20000.0,125.239300,54.953936,29.0,79.000,125.000,172.00,221.00
NO_OF_L_CR_TXNS,20000.0,12.348050,12.151921,0.0,6.000,10.000,14.00,75.00
NO_OF_BR_CSH_WDL_DR_TXNS,20000.0,1.883000,2.263596,0.0,1.000,1.000,2.00,15.00
NO_OF_ATM_DR_TXNS,20000.0,1.028950,1.497239,0.0,0.000,1.000,1.00,25.00
NO_OF_NET_DR_TXNS,20000.0,1.172450,2.404246,0.0,0.000,0.000,1.00,22.00


<p>Looking at all the columns with type='object', all of them are cateorial while the ACC_OP_DATE is a DateTime type</p>

In [13]:
def ConvertColTo(fromType, toType):
    for col_name in main_data.columns:
        if main_data[col_name].dtype == fromType:
            if col_name != 'ACC_OP_DATE':
                main_data[col_name] = main_data[col_name].astype(toType)
            else:
                main_data[col_name] = pd.to_datetime(main_data[col_name])
        
    main_data.info()

ConvertColTo("object", "category")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   TARGET                    20000 non-null  int64         
 1   AGE                       20000 non-null  int64         
 2   GENDER                    20000 non-null  category      
 3   BALANCE                   20000 non-null  float64       
 4   OCCUPATION                20000 non-null  category      
 5   SCR                       20000 non-null  int64         
 6   HOLDING_PERIOD            20000 non-null  int64         
 7   ACC_TYPE                  20000 non-null  category      
 8   ACC_OP_DATE               20000 non-null  datetime64[ns]
 9   LEN_OF_RLTN_IN_MNTH       20000 non-null  int64         
 10  NO_OF_L_CR_TXNS           20000 non-null  int64         
 11  NO_OF_BR_CSH_WDL_DR_TXNS  20000 non-null  int64         
 12  NO_OF_ATM_DR_TXNS 

In [14]:
'''
Input:
N/A

Output:
Go through each categorial column and print unique values for that column.

Retunrs:
N/A
'''
def CountUniqueValues():
    col_names = main_data.select_dtypes(include=['category']).columns.tolist()
    print(col_names)
    for col_name in col_names:
        print(f"\n======Unique Values count for: '{col_name}'========\n")
        for unique_col_value in  main_data[col_name].unique().tolist():
            total_count = main_data[col_name].count()
            unique_values_count = main_data[main_data[col_name] == unique_col_value][col_name].count()
            percentage = str(round((unique_values_count/total_count) * 100, 2))
            print(f"{unique_col_value} \t: {unique_values_count} ({percentage}%)")
        print(f"\n====================================================\n")


CountUniqueValues()

['GENDER', 'OCCUPATION', 'ACC_TYPE']

======Unique Values count for: 'GENDER'========

M 	: 14376 (71.88%)
F 	: 5433 (27.16%)
O 	: 191 (0.96%)



======Unique Values count for: 'OCCUPATION'========

SELF-EMP 	: 3568 (17.84%)
SAL 	: 5855 (29.28%)
PROF 	: 5417 (27.08%)
SENP 	: 5160 (25.8%)



======Unique Values count for: 'ACC_TYPE'========

SA 	: 15759 (78.8%)
CA 	: 4241 (21.2%)




In [15]:
'''
Input:
Axis, column name/x-axis, hue name

Output:
Displays a count plot.

Retunrs:
N/A
''' 
def HistBoxplot(box_chart_ax, hist_chart_ax, x_axis):
    sns.boxplot(
        data=main_data,
        x=main_data[x_axis],
        showmeans=True,
        ax=box_chart_ax)
    
    sns.histplot(
        data=main_data,
        x=main_data[x_axis],
        kde=True,
        ax=hist_chart_ax)
    
    hist_chart_ax.axvline(main_data[x_axis].mean(),                 # Get the mean of the values in the given column and draw a vertical line that cuts the chart on the mean value 
               color=stats_colors['Mean'],                                  # Use on of the colors predefined on this notebook
               label='Mean',                                                # Set the label to be diplayed on the legend
               linestyle="dashed");                                         # Make the line have dashes
    
    hist_chart_ax.axvline(main_data[x_axis].median(),               # Plot the median line on the chart.
               color=stats_colors['Median'],                                # Use on of the colors predefined on this notebook
               label='Median',                                              # Set the label to be diplayed on the legend
               linestyle="dashed");                                         # Make the line have dashes
    
    hist_chart_ax.axvline(main_data[x_axis].mode()[0],              # Plot the mode line on the chart.
               color=stats_colors['Mode'],                                  # Use on of the colors predefined on this notebook 
               label='Mode',                                                # Set the label to be diplayed on the legend
               linestyle="dashed");                                         # Make the line have dashes
    
    hist_chart_ax.legend(loc='upper right')

def PlotHistBoxGrid():
    col_names = main_data.select_dtypes(include=['float', 'int64']).columns.tolist()
    print(col_names)

    col_index = 0
    for r in range(0, int(len(col_names)), 3):
        fig, (box, hist) = plt.subplots(
            nrows=2,                                                                # Number of rows of the grid
            ncols=3,                                                                # Number of columns of the grid.
            figsize=(15,4),
            gridspec_kw={"height_ratios" : (0.25,0.5)},
            constrained_layout=True)

        for index in range(0, 3):
            if col_index < int(len(col_names)):
                HistBoxplot(box.flat[index], hist.flat[index], col_names[col_index])
            col_index += 1
            
'''
Input:
Column name

Output:
A series containing interquatile range values

Retunrs:
A series containing interquatile range values
''' 
def Get_IQR(col_name):
    quartiles = np.quantile(main_data[col_name][main_data[col_name].notnull()], [.2, .75])
    power_4iqr = 1 * (quartiles[1] - quartiles[0])
    print(f'Q1 = {quartiles[0]}, Q3 = {quartiles[1]}, 4*IQR = {power_4iqr}')
    outlier_powers = main_data.loc[np.abs(main_data[col_name] - main_data[col_name].median()) > power_4iqr, col_name]
    print(outlier_powers.array) 
    
    return quartiles

In [ ]:
PlotHistBoxGrid()

In [ ]:
fig, ax = plt.subplots(figsize=(18,18)) 
sns.heatmap(data=main_data.corr(), annot=True, linewidths=.5, ax=ax)
plt.show()

In [ ]:
main_data.drop(['AGE', 'FLG_HAS_ANY_CHGS', 'FLG_HAS_NOMINEE', 'FLG_HAS_OLD_LOAN', 'HOLDING_PERIOD', 'LEN_OF_RLTN_IN_MNTH', 'AMT_BR_CSH_WDL_DR'], axis=1, inplace=True)
main_data.info()

In [ ]:
# replace columns with  integers in the appropreate order
# new_feature_cols = {
#     "TXNS_COLS" : ['NO_OF_L_CR_TXNS', 'NO_OF_BR_CSH_WDL_DR_TXNS', 'NO_OF_ATM_DR_TXNS', 'NO_OF_NET_DR_TXNS', 'NO_OF_MOB_DR_TXNS', 'NO_OF_CHQ_DR_TXNS'],
#     "DR_COLS"   : ['AMT_ATM_DR', 'AMT_CHQ_DR', 'AMT_NET_DR', 'AMT_MOB_DR']
# }
# replace_struct = {
#     "NO_OF_TXNS"    : main_data[new_feature_cols['TXNS_COLS']].mean(axis=1),
#     "AMT_DR"        : main_data[new_feature_cols['DR_COLS']].mean(axis=1)
# }

# main_data['NO_OF_TXNS'] = main_data[new_feature_cols['TXNS_COLS']].mean(axis=1)
# main_data['AMT_DR']     = main_data[new_feature_cols['DR_COLS']].mean(axis=1)

# main_data.drop(new_feature_cols['TXNS_COLS'], axis=1, inplace=True)
# main_data.drop(new_feature_cols['DR_COLS'], axis=1, inplace=True)

# main_data.head(10)

In [ ]:
main_data['ACC_AGE_IN_MONTHS'] = ((datetime.now().year - main_data['ACC_OP_DATE'].dt.year ) * 12) - abs((main_data['ACC_OP_DATE'].dt.month - datetime.now().month))

main_data.drop("ACC_OP_DATE", axis=1, inplace=True)
main_data.head(10)

In [ ]:
# PlotHistBoxGrid()

In [ ]:
# col_names = ['NO_OF_TXNS', 'AMT_DR']

# for col_name in col_names:
#     print(f"\n\n==========================={col_name}====================\n")
#     main_data[col_name] = np.where(
#         main_data[col_name] < Get_IQR(col_name)[0],
#         main_data[col_name].mode(),
#         main_data[col_name])
#     print(f"\n\n==========================={col_name}====================\n")

In [ ]:
# PlotHistBoxGrid()

In [ ]:
fig, ax = plt.subplots(figsize=(18,18)) 
sns.heatmap(data=main_data.corr(), annot=True, linewidths=.5, ax=ax)
plt.show()

In [ ]:
# main_data.drop('ACC_AGE_IN_MONTHS', axis=1, inplace=True)

In [ ]:
# col_names = ['NO_OF_TXNS', 'AMT_DR']
# for col_name in col_names:
#     print(f"==========================={col_name}====================")
#     main_data[col_name + "_log"] = np.log(main_data[col_name])
#     main_data.drop([col_name], axis=1, inplace=True)
    
# PlotHistBoxGrid()

In [ ]:
# fig, ax = plt.subplots(figsize=(18,18)) 
# sns.heatmap(data=main_data.corr(), annot=True, linewidths=.5, ax=ax)
# plt.show()

In [ ]:
oneHotCols = main_data.select_dtypes(include=['category']).columns.tolist()
main_data = pd.get_dummies(main_data, columns=oneHotCols, drop_first=True)

main_data.head(10)

In [ ]:
# fig, ax = plt.subplots(figsize=(18,18)) 
# sns.heatmap(data=main_data.corr(), annot=True, linewidths=.5, ax=ax)
# plt.show()

In [ ]:
# sns.pairplot(main_data, kind="reg")

####  <span style="font-family: Arial; font-weight:bold;font-size:1.9em;color:#0e92ea"> 3. Building Decision Tree Model and Analysis:

In [ ]:
X = main_data.drop("TARGET", axis=1)
Y = main_data.pop("TARGET")
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

In [ ]:
feature_names = list(X.columns)
print(feature_names)

In [ ]:
# Functions to calculate different metrics

def DisplayConfusionMatrix(model, y_actual, labels=[1,0]):
    y_predict = model.predict(x_test)
    confusion_matrix = metrics.confusion_matrix(y_actual, y_predict, labels=[0, 1])
    confusion_matix_df = pd.DataFrame(
        confusion_matrix,
        index   = [i for i in ["Actual No", "Actual Yes"]],
        columns = [i for i in ["Predicted - No", "Predicted - Yes"]])
    
    group_counts = ["{0:0.0f}".format(value) for value in confusion_matrix.flatten()]
    group_percetages = ["{0:.2%}".format(value) for value in confusion_matrix.flatten()/np.sum(confusion_matrix)]
    
    labels = [f"{v1}\n{v2}" for v1, v2 in zip(group_counts, group_percetages)]
    labels = np.array(labels).reshape(2, 2)
    plt.figure(figsize= (10,7))
    sns.heatmap(confusion_matix_df, annot=labels,fmt='')
    plt.ylabel("True Label")
    plt.xlabel("Predicted Label")
    
def GetMetricsScore(model):
    
    pred_train = model.predict(x_train)
    pred_test  = model.predict(x_test)
    
    train_accuracy = model.score(x_train, y_train)
    test_accuracy  = model.score(x_test, y_test)
    
    train_recall = metrics.recall_score(y_train, pred_train)
    test_recall  = metrics.recall_score(y_test, pred_test)
    
    train_precision = metrics.precision_score(y_train, pred_train)
    test_precision  = metrics.precision_score(y_test, pred_test)

    f1_score_train = 2 * ((train_precision * train_recall)/(train_precision + train_recall))
    f1_score_test = 2 * ((test_precision * test_recall)/(test_precision + test_recall))
    
    return {
        'Accuracy_Test'   : test_accuracy,
        'Accuracy_Train'  : train_accuracy,
        'Recall_Test'     : test_recall,
        'Recall_Train'    : train_recall,
        'Precision_Test'  : test_precision,
        'Precision_Train' : train_precision,
        'F1_Score_Train'  : f1_score_train,
        'F1_Score_Test'   : f1_score_test
    }
        

def DisplayImportance(model):
    gini_importances = pd.DataFrame(model.feature_importances_, columns=["Imp"], index=x_train.columns).sort_values(by = "Imp")
    print(gini_importances)
    
def DisplayImportanceChart(model):
    importances = model.feature_importances_
    indices = np.argsort(importances)

    plt.figure(figsize=(12, 12))
    plt.title('Feature Importance')
    plt.barh(range(len(indices)), importances[indices], color='violet', align='center')
    plt.yticks(range(len(indices)), [list(X.columns)[i] for i in indices])
    plt.xlabel('Relative Importance')
    plt.show()
    
def PlotTree(model, size = [20, 30]):
    plt.figure(figsize=(size[0],size[1]))
    tree.plot_tree(model, feature_names=feature_names, filled=True, fontsize=9, node_ids=True, class_names=True)
    plt.show()
    
def PrintCoeficients(model):
    coefs = pd.DataFrame(
        np.append(model.coef_, model.intercept_),
        index=x_train.columns.tolist() + ["Intercept"],
        columns=["Coefficients"],
    )

    coefs.sort_values('Coefficients')

In [ ]:
dTree_Model = DecisionTreeClassifier(criterion= 'gini', random_state=1)
dTree_Model.fit(x_train, y_train)

In [ ]:
PlotTree(dTree_Model)

In [ ]:
DisplayConfusionMatrix(dTree_Model, y_test)

In [ ]:
dTree_model_score = GetMetricsScore(dTree_Model)
pprint.pprint(dTree_model_score)

In [ ]:
DisplayImportanceChart(dTree_Model)

In [ ]:

# choose the type of classifier
dTree_Model = DecisionTreeClassifier(random_state=1)

# Grid of params to choose from
parameters = {
    "max_depth": np.arange(1, 10),
    "min_samples_leaf": np.arange(1, 20),
    "max_leaf_nodes" : np.arange(1, 15),
    "min_impurity_decrease": [0.001, 0.01, 0.1]
}

# type of score used to compare paramter combinations
acc_scorer =metrics.make_scorer(metrics.recall_score)

# run the grid search
grid_obj = GridSearchCV(dTree_Model, parameters, scoring=acc_scorer, cv=5)
grid_obj = grid_obj.fit(x_train, y_train)

# set the clf to the best combination of parameters
dTree_Model = grid_obj.best_estimator_

# fit the best algorithm to the data
dTree_Model.fit(x_train, y_train)

In [ ]:
PlotTree(dTree_Model, [15, 10])

In [ ]:
dTree_model_score = GetMetricsScore(dTree_Model)
pprint.pprint(dTree_model_score)

In [ ]:
DisplayConfusionMatrix(dTree_Model, y_test)

In [ ]:
DisplayImportanceChart(dTree_Model)

####  <span style="font-family: Arial; font-weight:bold;font-size:1.9em;color:#0e92ea"> 4. Building Logistic Regression Model and Analysis:

In [ ]:
# fit model on train
logistic_reg_model = LogisticRegression()
logistic_reg_model.fit(x_train,y_train)

# predict on test
y_predict = logistic_reg_model.predict(x_test)

In [ ]:
PrintCoeficients(logistic_reg_model)

In [ ]:
logistic_reg_model_score = GetMetricsScore(logistic_reg_model)
pprint.pprint(logistic_reg_model_score)

In [ ]:
DisplayConfusionMatrix(logistic_reg_model, y_test)